In [ ]:
#The 3 architectures above emerged after hundreds of trials and hypertuning tests with the use of Optuna framework

In [ ]:
X_train, y_train, X_test_countries, y_test_countries, X_val, y_val = get_data()
inp = Input(shape=(np.array(X_train).shape[1], np.array(X_train).shape[2], ))
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, decay_steps=40000, decay_rate=0.9)

callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)

callback2 = StepCosineDecayRestartCallback(initial_learning_rate=1e-4,
                                          first_decay_steps=20,
                                          t_mul=2,
                                          m_mul=1,
                                          jump_step=1)

class HistorySaveValueCallback(tf.keras.callbacks.Callback):
    """Saves the value to the Model's History object"""

    def __init__(self, name: str, get_value_func):
        self.get_value_func = get_value_func
        self.name = name

    def on_epoch_end(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        logs[self.name] = float(tf.keras.backend.get_value(self.get_value_func(self)))

callback3 = HistorySaveValueCallback(name='lr', get_value_func=lambda x: x.model.optimizer.learning_rate)

In [ ]:
#Pretrained model 1
x = Conv1D(filters=128, kernel_size=3, activation='relu')(inp)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = Conv1D(filters=104, kernel_size=6, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(50)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
out = Dense(24)(x)

model1 =  Model(inp, out)
model1.compile(optimizer=Adam(), loss='mse')
history = model1.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), epochs=200, callbacks=[callback1, callback2, callback3], batch_size=2048)

In [ ]:
predictions_per_country = []
inversed_y_per_country = []
results = []
for i in range(len(scalers)):
    predict = model1.predict(np.array(X_test_countries[i]))
    predictions_per_country.append(scalers[i].inverse_transform(predict))
    inversed_y_per_country.append(scalers[i].inverse_transform(y_test_countries[i]))
    print("Model 1 RMSE for country ", i, ":" , mean_squared_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))
    print("Model 1 MAE for country ", i, ":" , mean_absolute_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))

In [31]:
#Pretrained model 2

In [32]:
x = Conv1D(filters=72, kernel_size=3, activation='relu')(inp)
x = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x)
x1 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x)
x1 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x1)
x1 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x1)
x = Add()([x, x1])
x2 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x)
x2 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x2)
x2 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x2)
x2 = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x2)
x = Add()([x, x2])
x = Conv1D(filters=104, kernel_size=4, activation='relu',padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(250, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(250, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(250, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(250)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(250, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(250, activation='relu')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
out = Dense(24, activation='relu')(x)

model2 =  Model(inp, out)
model2.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mse')
history = model2.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), epochs=200, callbacks=[callback1, callback2, callback3], batch_size=2048)

Epoch 1/200
131/131 [==============================] - 350s 2s/step - loss: 0.0459 - val_loss: 0.0337 - lr: 0.0100
Epoch 2/200
131/131 [==============================] - 359s 3s/step - loss: 0.0157 - val_loss: 0.0134 - lr: 0.0100
Epoch 3/200
131/131 [==============================] - 374s 3s/step - loss: 0.0140 - val_loss: 0.0268 - lr: 0.0100
Epoch 4/200
131/131 [==============================] - 376s 3s/step - loss: 0.0126 - val_loss: 0.0276 - lr: 0.0100
Epoch 5/200
131/131 [==============================] - 374s 3s/step - loss: 0.0128 - val_loss: 0.0176 - lr: 0.0100
Epoch 6/200
131/131 [==============================] - 371s 3s/step - loss: 0.0093 - val_loss: 0.0258 - lr: 0.0100
Epoch 7/200
131/131 [==============================] - 372s 3s/step - loss: 0.0081 - val_loss: 0.0150 - lr: 0.0100
Epoch 8/200
131/131 [==============================] - 377s 3s/step - loss: 0.0078 - val_loss: 0.0077 - lr: 0.0100
Epoch 9/200
131/131 [==============================] - 377s 3s/step - loss: 0.00

In [ ]:
predictions_per_country = []
inversed_y_per_country = []
results = []
for i in range(len(scalers)):
    predict = model2.predict(np.array(X_test_countries[i]))
    predictions_per_country.append(scalers[i].inverse_transform(predict))
    inversed_y_per_country.append(scalers[i].inverse_transform(y_test_countries[i]))
    print("Model 2 RMSE for country ", i, ":" , mean_squared_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))
    print("Model 2 MAE for country ", i, ":" , mean_absolute_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))

In [ ]:
x = Conv1D(filters=104, kernel_size=3, activation='relu', padding='same')(inp)
x = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x)
x1 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x)
x1 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x1)
x1 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x1)
x = Add()([x, x1])
x2 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x)
x2 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x2)
x2 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x2)
x2 = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x2)
x = Add()([x, x2])
x = Conv1D(filters=104, kernel_size=2, activation='relu', padding='same')(x)
x = MaxPooling1D()(x)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(150)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(150)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Dense(150)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
out = Dense(24)(x)

model3 = Model(inp, out)
model3.compile(optimizer=Adam(learning_rate=1e-2), loss='mse')
history = model3.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), epochs=200, callbacks=[callback1, callback2, callback3], batch_size=1024)

In [ ]:
predictions_per_country = []
inversed_y_per_country = []
results = []
for i in range(len(scalers)):
    predict = model3.predict(np.array(X_test_countries[i]))
    predictions_per_country.append(scalers[i].inverse_transform(predict))
    inversed_y_per_country.append(scalers[i].inverse_transform(y_test_countries[i]))
    print("Model 3 RMSE for country ", i, ":" , mean_squared_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))
    print("Model 3 MAE for country ", i, ":" , mean_absolute_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))